In [42]:
import pandas as pd

# Load datasets
songs = pd.read_csv('../data/songs.csv')
streams = pd.read_csv('../data/streams1.csv')
users = pd.read_csv('../data/users.csv')

In [3]:
# Prepare data
streams['listen_date'] = pd.to_datetime(streams['listen_time']).dt.date
streams['listen_hour'] = pd.to_datetime(streams['listen_time']).dt.hour
full_data = streams.merge(songs, on='track_id').merge(users, on='user_id')
full_data.head()

,user_id,track_id,listen_time,listen_date,listen_hour,id,artists,album_name,track_name,popularity,...,instrumentalness,liveness,valence,tempo,time_signature,track_genre,user_name,user_age,user_country,created_at
0,26213,4dBa8T7oDV9WvGr7kVS4Ez,2024-06-25 17:43:13,2024-06-25,17,91162,Panic! At The Disco,Pop n' Fresh,Don't Let the Light Go Out,0,...,0.00230,0.271,0.1790,117.637,4,rock,Cathy Smith,39,United States,2024-06-05
1,6937,4osgfFTICMkcGbbigdsa53,2024-06-25 07:26:00,2024-06-25,7,103402,Frank Ocean,Novacane,Novacane,81,...,0.00126,0.160,0.3700,93.510,4,soul,Tommy Grant,61,United States,2024-10-16
2,21407,2LoQWx41KeqOrSFra089YS,2024-06-25 13:25:26,2024-06-25,13,42552,Internal Rot,Grieving Birth,Harpooned,14,...,0.71800,0.500,0.0689,122.842,3,grindcore,Garrett Ryan,34,United States,2024-08-05
3,47146,7cfG5lFeJWEgpSnubt4O4W,2024-06-25 18:17:50,2024-06-25,18,50935,Rata Blanca,The forgotten Kingdom,The Forgotten Kingdom,20,...,0.00434,0.184,0.5230,130.064,4,heavy-metal,Patrick Nash,44,United States,2024-01-21
4,47146,7KHG8MHzxAv0YnLFBWY9Y1,2024-06-25 18:17:48,2024-06-25,18,76716,Jean-Baptiste Lully;Mary Enid Haines;Sharla Na...,Lully: Ballet Music for the Sun King,Le bourgeois gentilhomme: Chaconne des Scaramo...,51,...,0.01970,0.138,0.0000,0.000,0,opera,Patrick Nash,44,United States,2024-01-21


In [4]:
# KPI 1: Hourly Unique Listeners
hourly_unique_listeners = full_data.groupby(['listen_date', 'listen_hour'])['user_id'].nunique().reset_index(name='unique_listeners')
hourly_unique_listeners.head()

,listen_date,listen_hour,unique_listeners
0,2024-06-25,0,376
1,2024-06-25,1,383
2,2024-06-25,2,356
3,2024-06-25,3,369
4,2024-06-25,4,379


In [5]:
# KPI 2: Top Listened Artist of the Hour
artist_listen_counts = full_data.groupby(['listen_date', 'listen_hour', 'artists']).size().reset_index(name='listen_counts')
artist_listen_counts.head()

,listen_date,listen_hour,artists,listen_counts
0,2024-06-25,0,347aidan,1
1,2024-06-25,0,A.N.I.M.A.L.;Chuck Jhonson and Juan Jose Burgo...,1
2,2024-06-25,0,AKB48,2
3,2024-06-25,0,ATTLAS,1
4,2024-06-25,0,Aaron Kwok,1


In [13]:
#artist_listen_counts.groupby(['listen_date', 'listen_hour'])['listen_counts'].idxmax()
top_artist = artist_listen_counts.loc[artist_listen_counts.groupby(['listen_date', 'listen_hour'])['listen_counts'].idxmax()]
top_artist.head()

,listen_date,listen_hour,artists,listen_counts
159,2024-06-25,0,Hank Williams,4
503,2024-06-25,1,BTS,3
1014,2024-06-25,2,Dani Fernández,3
1401,2024-06-25,3,Apollo 440,3
1965,2024-06-25,4,George Jones,5


In [15]:
# KPI 2: Top Listened Artist of the Hour
artist_listen_counts = full_data.groupby(['listen_date', 'listen_hour', 'artists']).size().reset_index(name='listen_counts')
top_artist = artist_listen_counts.loc[artist_listen_counts.groupby(['listen_date', 'listen_hour'])['listen_counts'].idxmax()]
top_artist = top_artist.rename(columns={'artists': 'top_artist'})
top_artist.head()

,listen_date,listen_hour,top_artist,listen_counts
159,2024-06-25,0,Hank Williams,4
503,2024-06-25,1,BTS,3
1014,2024-06-25,2,Dani Fernández,3
1401,2024-06-25,3,Apollo 440,3
1965,2024-06-25,4,George Jones,5


In [16]:
# KPI 3: Listening Sessions per User per Hour
full_data['session_id'] = full_data['user_id'].astype(str) + '-' + full_data['listen_time'].astype(str)
sessions_per_user = full_data.groupby(['listen_date', 'listen_hour', 'user_id']).nunique('session_id').reset_index()
avg_sessions_per_user = sessions_per_user.groupby(['listen_date', 'listen_hour'])['session_id'].mean().reset_index(name='avg_sessions_per_user')
avg_sessions_per_user.head()

,listen_date,listen_hour,avg_sessions_per_user
0,2024-06-25,0,1.289894
1,2024-06-25,1,1.234987
2,2024-06-25,2,1.311798
3,2024-06-25,3,1.276423
4,2024-06-25,4,1.229551


In [29]:
# KPI 4: Hourly Track Diversity Index
#full_data.head()
#full_data.groupby(['listen_date', 'listen_hour'])['track_id'].agg(['nunique', 'count'])
track_diversity = full_data.groupby(['listen_date', 'listen_hour'])['track_id'].agg(['nunique', 'count']).reset_index()
#track_diversity.head()
track_diversity['diversity_index'] = track_diversity['nunique'] / track_diversity['count']
track_diversity.head()

,listen_date,listen_hour,nunique,count,diversity_index
0,2024-06-25,0,484,485,0.997938
1,2024-06-25,1,471,473,0.995772
2,2024-06-25,2,465,467,0.995717
3,2024-06-25,3,468,471,0.993631
4,2024-06-25,4,464,466,0.995708


In [39]:
# KPI 5: Most Engaged User Group by Age per Hour
# Assuming age groups are segmented as 18-25, 26-35, etc.
#pd.cut(users['user_age'], bins=[0, 25, 35, 45, 55, 65, 100], labels=['18-25', '26-35', '36-45', '46-55', '56-65', '66+'])
users['age_group'] = pd.cut(users['user_age'], bins=[0, 25, 35, 45, 55, 65, 100], labels=['18-25', '26-35', '36-45', '46-55', '56-65', '66+'])
#full_data.merge(users, on='user_id').groupby(['listen_date', 'listen_hour', 'age_group']).size().reset_index(name='streams')
user_group_engagement = full_data.merge(users, on='user_id').groupby(['listen_date', 'listen_hour', 'age_group']).size().reset_index(name='streams')
#user_group_engagement.head()
user_group_engagement.groupby(['listen_date', 'listen_hour'])['streams'].idxmax()
most_engaged_group = user_group_engagement.loc[user_group_engagement.groupby(['listen_date', 'listen_hour'])['streams'].idxmax()].rename(columns={'age_group': 'most_engaged_age_group'})
most_engaged_group

C:\Users\jampa\AppData\Local\Temp\ipykernel_40296\344694032.py:6: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  user_group_engagement = full_data.merge(users, on='user_id').groupby(['listen_date', 'listen_hour', 'age_group']).size().reset_index(name='streams')


,listen_date,listen_hour,most_engaged_age_group,streams
3,2024-06-25,0,46-55,109
8,2024-06-25,1,36-45,114
16,2024-06-25,2,56-65,98
21,2024-06-25,3,46-55,99
25,2024-06-25,4,26-35,98
33,2024-06-25,5,46-55,109
37,2024-06-25,6,26-35,95
43,2024-06-25,7,26-35,103
50,2024-06-25,8,36-45,104
57,2024-06-25,9,46-55,118


In [43]:
# Prepare data
streams['listen_date'] = pd.to_datetime(streams['listen_time']).dt.date
streams['listen_hour'] = pd.to_datetime(streams['listen_time']).dt.hour
full_data = streams.merge(songs, on='track_id').merge(users, on='user_id')

# KPI 1: Hourly Unique Listeners
hourly_unique_listeners = full_data.groupby(['listen_date', 'listen_hour'])['user_id'].nunique().reset_index(name='unique_listeners')

# KPI 2: Top Listened Artist of the Hour
artist_listen_counts = full_data.groupby(['listen_date', 'listen_hour', 'artists']).size().reset_index(name='listen_counts')
top_artist = artist_listen_counts.loc[artist_listen_counts.groupby(['listen_date', 'listen_hour'])['listen_counts'].idxmax()]
top_artist = top_artist.rename(columns={'artists': 'top_artist'})

# KPI 3: Listening Sessions per User per Hour
full_data['session_id'] = full_data['user_id'].astype(str) + '-' + full_data['listen_time'].astype(str)
sessions_per_user = full_data.groupby(['listen_date', 'listen_hour', 'user_id']).nunique('session_id').reset_index()
avg_sessions_per_user = sessions_per_user.groupby(['listen_date', 'listen_hour'])['session_id'].mean().reset_index(name='avg_sessions_per_user')

# KPI 4: Hourly Track Diversity Index
track_diversity = full_data.groupby(['listen_date', 'listen_hour'])['track_id'].agg(['nunique', 'count']).reset_index()
track_diversity['diversity_index'] = track_diversity['nunique'] / track_diversity['count']

# KPI 5: Most Engaged User Group by Age per Hour
# Assuming age groups are segmented as 18-25, 26-35, etc.
users['age_group'] = pd.cut(users['user_age'], bins=[0, 25, 35, 45, 55, 65, 100], labels=['18-25', '26-35', '36-45', '46-55', '56-65', '66+'])
user_group_engagement = full_data.merge(users, on='user_id').groupby(['listen_date', 'listen_hour', 'age_group']).size().reset_index(name='streams')
most_engaged_group = user_group_engagement.loc[user_group_engagement.groupby(['listen_date', 'listen_hour'])['streams'].idxmax()].rename(columns={'age_group': 'most_engaged_age_group'})

# Combine all KPIs into one DataFrame
final_kpis = hourly_unique_listeners.merge(top_artist, on=['listen_date', 'listen_hour'])
final_kpis = final_kpis.merge(avg_sessions_per_user, on=['listen_date', 'listen_hour'])
final_kpis = final_kpis.merge(track_diversity[['listen_date', 'listen_hour', 'diversity_index']], on=['listen_date', 'listen_hour'])
final_kpis = final_kpis.merge(most_engaged_group[['listen_date', 'listen_hour', 'most_engaged_age_group']], on=['listen_date', 'listen_hour'])


C:\Users\jampa\AppData\Local\Temp\ipykernel_40296\46502605.py:26: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  user_group_engagement = full_data.merge(users, on='user_id').groupby(['listen_date', 'listen_hour', 'age_group']).size().reset_index(name='streams')


In [44]:
# Display the combined DataFrame
final_kpis

,listen_date,listen_hour,unique_listeners,top_artist,listen_counts,avg_sessions_per_user,diversity_index,most_engaged_age_group
0,2024-06-25,0,376,Hank Williams,4,1.289894,0.997938,46-55
1,2024-06-25,1,383,BTS,3,1.234987,0.995772,36-45
2,2024-06-25,2,356,Dani Fernández,3,1.311798,0.995717,56-65
3,2024-06-25,3,369,Apollo 440,3,1.276423,0.993631,46-55
4,2024-06-25,4,379,George Jones,5,1.229551,0.995708,26-35
5,2024-06-25,5,372,AJR,2,1.263441,0.997872,46-55
6,2024-06-25,6,362,Exaltasamba,3,1.273481,1.000000,26-35
7,2024-06-25,7,372,Sorriso Maroto,3,1.287634,1.000000,26-35
8,2024-06-25,8,375,At The Gates,2,1.277333,1.000000,36-45
9,2024-06-25,9,417,Arctic Monkeys,4,1.256595,0.998092,46-55
